In [9]:
import tensorflow as tf
import time
import numpy as np
import pandas as pd

# Load data

train_file_path_custom='C:/Users/Ajay/Downloads/human_annotations.tsv'

with open(train_file_path_custom, 'r', encoding='utf-8') as file:
    data_custom = file.readlines()

data_split_custom = [line.strip().split('\t') for line in data_custom]

df = pd.DataFrame(data_split_custom, columns=['language_id', 'value', 'Source', 'Indic_Sentence',
                                                      'English_Sentence', 'Labse_Score', 'bucket'])
train_df = df[df['language_id'] == 'Hindi'][1:]

# Preprocess sentences
eng = train_df['English_Sentence'].apply(lambda x: f"<SOS> {x} <EOS>")
hind = train_df['Indic_Sentence'].apply(lambda x: f"<SOS> {x} <EOS>")

# Tokenize sentences
eng_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=string.punctuation, oov_token='<unk>')
hind_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=string.punctuation, oov_token='<unk>')
eng_tokenizer.fit_on_texts(eng)
hind_tokenizer.fit_on_texts(hind)

# Create input sequences
inputs = eng_tokenizer.texts_to_sequences(eng)
targets = hind_tokenizer.texts_to_sequences(hind)

# Pad sequences
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, padding='post', truncating='post')

# Define Transformer model
class Transformer(tf.keras.Model):
    def _init_(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self)._init_()
        self.encoder = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.decoder = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.transformer = tf.keras.Sequential([
            tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model),
            tf.keras.layers.Dropout(rate),
            tf.keras.layers.LayerNormalization(epsilon=1e-6),
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(rate),
            tf.keras.layers.LayerNormalization(epsilon=1e-6),
        ], name='encoder_layer')
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, targets, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inputs)
        dec_output = self.decoder(targets)
        for layer in self.transformer.layers:
            dec_output = layer(dec_output)
        final_output = self.final_layer(dec_output)
        return final_output

# Set hyperparameters
d_model, num_heads, dff = 128, 8, 512
input_vocab_size, target_vocab_size = len(eng_tokenizer.word_index) + 1, len(hind_tokenizer.word_index) + 1
pe_input, pe_target = 1000, 1000
num_layers, rate = 4, 0.1

# Create transformer model
transformer = Transformer(num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate)

# Compile model
transformer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
transformer.fit(inputs, targets, epochs=5, batch_size=64)

ValueError: Models passed to `fit` can only have `training` and the first argument in `call()` as positional arguments, found: ['targets', 'enc_padding_mask', 'look_ahead_mask', 'dec_padding_mask'].